In [18]:
# Local application imports
from utils import postgresListDatabases, postgresListTables, postgresImportFile, postgresRunSqlScript

In [19]:
db_name='weather_wizards'

sql_file = 'sql/schema.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

sql_file = 'sql/swdiws_nx3tvs.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

table_name = 'swdiws_nx3tvs_geojson'
file_location = 'data/swdiws_nx3tvs_geojson.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")     

table_name = 'radar_station'
file_location = 'data/radar_locations.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

databases = postgresListDatabases()
for database in databases:
    print(f"{database}")
    tables = postgresListTables(database)
    for table in tables:
        print(f"\t{table}")

ERROR:root:Error executing SQL script sql/schema.sql: FATAL:  password authentication failed for user "manahil1"

ERROR:root:Error executing SQL script sql/swdiws_nx3tvs.sql: FATAL:  password authentication failed for user "manahil1"

ERROR:root:Error importing file data/swdiws_nx3tvs_geojson.csv into table swdiws_nx3tvs_geojson: FATAL:  password authentication failed for user "manahil1"

ERROR:root:Error importing file data/radar_locations.csv into table radar_station: FATAL:  password authentication failed for user "manahil1"

ERROR:root:An error occurred while retrieving databases: FATAL:  password authentication failed for user "manahil1"



An error occurred while executing the SQL script: FATAL:  password authentication failed for user "manahil1"

An error occurred while executing the SQL script: FATAL:  password authentication failed for user "manahil1"

Loaded 0 records to swdiws_nx3tvs_geojson
Loaded 0 records to radar_station


In [20]:
db_name='weather_wizards'
sql_file = 'sql/hurricane_name_data.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)


ERROR:root:Error executing SQL script sql/hurricane_name_data.sql: FATAL:  password authentication failed for user "manahil1"



An error occurred while executing the SQL script: FATAL:  password authentication failed for user "manahil1"



In [8]:

table_name = 'hurricane_name_data_staging'
file_location = 'data/hurricane_details_retired.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")     


Loaded 7917 records to hurricane_name_data_staging


In [1]:

table_name = 'hurricane_name_data_staging'
file_location = 'data/hurricane_details.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")   

NameError: name 'postgresImportFile' is not defined